# Particle Swarm Optimisation

This notebook showcases how to use the built-in Particle Swarm Optimisation (PSO) algorithm.

In [1]:
using EvoLP
using Statistics
using OrderedCollections

For this example, we will use the Michalewicz function:

In [2]:
@doc michalewicz

```
michalewicz(x; m=10)
```

The **Michalewicz** function is a $d$-dimensional function with several steep valleys, where `m` controls the steepness. `m` is usually set at 10. For 2 dimensions, $x^* = [2.20, 1.57]$, with $f(x^*) = -1.8011$.

$$
f(x) = -\sum_{i=1}^{d}\sin(x_i) \sin^{2m}\left(\frac{ix_i^2}{\pi}\right)
$$


In this case we will use $d=2$ and $m=10$, which is the default value implemented in EvoLP.

In PSO, we use _particles_. Each particle has a position and a velocity, and remembers the best position it has visited.

We can create a population of particles in multiple ways, but EvoLP provides 2 particle generators with random positions: either uniform or following a normal distribution.

Let's use the normal generator:

In [3]:
@doc normal_rand_particle_pop

```
normal_rand_particle_pop(n, μ, Σ; rng=Random.GLOBAL_RNG)
```

Generate a population of `n` [`Particle`](@ref) using a normal distribution with means `μ and covariance`Σ`.

`μ` expects a vector of length *l* (i.e. number of dimensions) while `Σ` expects an *l x l* matrix of covariances.

# Examples

```julia
julia> normal_rand_particle_pop(3, [-1, -1], [1 0; 0 1])
3-element Vector{Any}:
 Particle([-2.3026589618390214, 0.25907687184121864], [0.0, 0.0], [-2.3026589618390214, 0.25907687184121864])
 Particle([-0.5118786279984703, -0.5948648935657292], [0.0, 0.0], [-0.5118786279984703, -0.5948648935657292])
 Particle([-1.3230210847731094, -1.6234307114658497], [0.0, 0.0], [-1.3230210847731094, -1.6234307114658497])
```


In [4]:
population = normal_rand_particle_pop(50, [0, 0], [1 0; 0 1])
first(population, 3)

3-element Vector{Any}:
 Particle([-0.22703948747578281, 1.7689889087227626], [0.0, 0.0], [-0.22703948747578281, 1.7689889087227626])
 Particle([0.9523372333139276, 1.8452380648469366], [0.0, 0.0], [0.9523372333139276, 1.8452380648469366])
 Particle([-1.2560899837782407, 0.15303679468484374], [0.0, 0.0], [-1.2560899837782407, 0.15303679468484374])

Let's use the `Logbook` to save information about each iteration of the run:

In [5]:
@doc Logbook

```
Logbook(S::LittleDict)
```

A log for statistics intended for use on every iteration of an algorithm. The logbook is constructed from a `LittleDict` ordered dictionary which maps stat names (strings) to callables, such that *statname* $i$ can be computed from *callable* $i$.

The resulting `Logbook` contains:

  * `S::LittleDict`: The ordered dict of stat names and callables
  * `records::AbstractVector`: A vector of NamedTuples where each field is a statistic.


In [6]:
statnames = ["avg_fit", "median_fit", "best_fit"]
callables = [mean, median, minimum]

thedict = LittleDict(statnames, callables)
logbook = Logbook(thedict)

Logbook(LittleDict{AbstractString, Function, Vector{AbstractString}, Vector{Function}}("avg_fit" => Statistics.mean, "median_fit" => Statistics.median, "best_fit" => minimum), NamedTuple{(:avg_fit, :median_fit, :best_fit)}[])

We can now use the built-in algorithm:

In [7]:
@doc PSO

```
PSO(f::Function, population, k_max; w=1, c1=1, c2=1)
PSO(logger::Logbook, f::Function, population, k_max; w=1, c1=1, c2=1)
```

## Arguments

  * `f::Function`: Objective function to minimise
  * `population`: Population—a list of [`Particle`](@ref) individuals
  * `k_max`: maximum iterations
  * `w`: Inertia weight. Optional, by default 1.
  * `c1`: Cognitive coefficient (my position). Optional, by default 1
  * `c2`: Social coefficient (swarm position). Optional, by default 1

Returns a [`Result`](@ref).


In [8]:
results = PSO(logbook, michalewicz, population, 30);

The output was suppressed so that we can analyse each part of the result separately:

In [9]:
@show optimum(results)

@show optimizer(results)

@show iterations(results)
@show f_calls(results);

optimum(results) = -1.7945699649685944
optimizer(results) = Particle([2.4587719347604904, 1.5710645311634195], [-0.3065425967254033, 3.75884997817273e-5], [2.1824224667308583, 1.5710530344414129])
iterations(results) = 30
f_calls(results) = 4601


We can also take a look at the logbook's records and see how the statistics we calculated changed throughout the run:

In [10]:
for (i, I) in enumerate(logbook.records)
    print("it: $(i) with best_pos: $(I[3]) and avg_pos: $(I[1]) \n")
end

it: 1 with best_pos: -0.6906794768370129 and avg_pos: -0.005754793636862157 
it: 2 with best_pos: -0.9897012274595496 and avg_pos: -0.17476488948373003 
it: 3 with best_pos: -1.102946135488689 and avg_pos: -0.2783471388805291 
it: 4 with best_pos: -0.968940921732735 and avg_pos: -0.3095523155518831 
it: 5 with best_pos: -1.472368499905572 and avg_pos: -0.3379717198952197 
it: 6 with best_pos: -1.3055867468693856 and avg_pos: -0.3258715803445814 
it: 7 with best_pos: -1.3634607463646478 and avg_pos: -0.45466359334515916 
it: 8 with best_pos: -1.6705825784286839 and avg_pos: -0.6029634734635035 
it: 9 with best_pos: -1.5345866019317895 and avg_pos: -0.5501090751560626 
it: 10 with best_pos: -1.7212146343892145 and avg_pos: -0.5773626889249354 
it: 11 with best_pos: -1.779247279246615 and avg_pos: -0.6533233522934709 
it: 12 with best_pos: -1.7838823541946531 and avg_pos: -0.6378969618098259 
it: 13 with best_pos: -1.7931685047158967 and avg_pos: -0.6128555872381239 
it: 14 with best_pos: